<a href="https://colab.research.google.com/github/markvasin/deep_learning_exercise/blob/master/lab4/4_4_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 4: Using GPU acceleration with PyTorch

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 40.1MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [ ]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[-0.2833,  0.2216,  0.0347,  ...,  0.6017, -0.6966, -1.2116],
        [ 0.3087,  0.8155, -0.0906,  ..., -0.0528, -2.2587,  1.6875],
        [-0.4872,  2.5662, -1.6106,  ...,  0.0101, -0.7423,  0.7105],
        ...,
        [ 0.2633,  0.9001,  0.1193,  ...,  0.5986, -0.0479,  2.4303],
        [ 1.6265, -0.4990, -0.3565,  ...,  1.0421,  0.2790, -0.1675],
        [-0.0098, -0.5389, -0.1770,  ...,  0.5911, -0.1306,  0.0177]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [ ]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[ 1.0455e+00,  1.1920e-01,  1.8050e-02,  ...,  2.3479e-01,
         -1.4628e+00,  2.4844e-01],
        [-5.9708e-02, -7.4044e-01,  5.9224e-01,  ...,  7.3269e-02,
         -6.3229e-02, -7.0966e-02],
        [-6.5815e-01, -1.0227e+00, -5.3817e-02,  ..., -5.7058e-02,
          3.3035e-04,  2.0041e-01],
        ...,
        [ 7.8387e-01,  4.4302e-02, -5.5957e-01,  ...,  8.5790e-01,
         -5.2145e-01, -1.9081e-01],
        [ 3.0560e-01, -5.3264e-02,  5.6519e-01,  ...,  1.0483e-01,
          1.0277e+00, -9.0043e-02],
        [ 7.5061e-01, -6.6784e-01, -9.0654e-01,  ..., -1.8604e+00,
          6.5258e-01,  9.3116e-01]], device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-2.4679e-01,  7.0538e-01,  3.6574e-02,  ...,  1.7073e+00,
          1.2246e-01,  1.6897e-02],
        [-6.0381e-01,  2.6113e+00,  2.1042e-01,  ...,  9.9233e-02,
          1.9702e+00, -2.9189e-01],
        [-2.4672e-01,  2.0552e-01,  8.1623e-01,  ..., -3.3230e-01,
          5.5759e-01,  1.5719e-01],
        ...,
        [ 2.0760e+00,  6.0122e-01,  9.1589e-01,  ..., -1.0251e-01,
          3.2704e-01,  7.7485e-01],
        [ 2.3756e-01,  2.6425e-03, -6.9503e-01,  ...,  2.1971e-01,
         -3.9109e+00,  2.7339e-02],
        [-2.6727e-01,  3.7530e-01,  1.8196e-02,  ..., -4.9869e-01,
          1.0443e+00,  1.8498e-01]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

GPU has large number of cores, and are more bandwidth efficient compare to CPU.

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

GPU memory